In [69]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True);

memory.save_context({"input":"Hi!"}, {"output":"How are you?"})

memory.load_memory_variables({})

{'history': 'Human: Hi!\nAI: How are you?'}